This notebook reads in the cleaned OSC file (1.OSCCleaned.csv') and searches through ESO & MUSE map images, looking for OSC objects that have gone off in their footprint. 

It then converts the ra & dec coordinates to map pixels and measures the pixel intensity there.

This notebook prints the OSC Object data on the '2.SNeHaMasterCat.txt'. 

This notebook then turns the 2.SNeHaMasterCat.txt file into a dataframe, where the non SN objects are removed and then the cleaned table is printed to '2.SNeHaMasterCatClean.csv'.

This cleaned table is used to plot all of the galaxy images and their respective SNe in ../Figures/2.ESO_Galaxy_Images & 2.ESO_SNe_Galaxy_Images, ../Figures/2.MUSE_Galaxy_Images & 2.MUSE_SNe_Galaxy_Images

In [1]:
#import packages and formatting statements

import numpy as np
from matplotlib import pyplot as plt
import astropy.io.fits as pyfits
from astropy.table import Table, join, Column
from astropy.wcs import WCS
from astropy.io import ascii
import os

plt.rc('text',usetex=False)
fontsize=20
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)

In [2]:
# Compile galaxy and OSC objects
DataFile = '../Data/1.ProjectTable.csv'
Data = Table.read(DataFile, format='csv') 
galaxies, ESOmaps, MUSEres, MUSEmaps = Data["Galaxy"], Data["ESO-MPG_Hasub_wcomb_corr"], Data["MUSEResolution"], Data["MUSEmap"]
SNe = np.loadtxt('../Data/1.OSCRaDeg.txt', dtype = str, unpack=True, usecols = 0)
ras = np.loadtxt('../Data/1.OSCRaDeg.txt',delimiter='" ', unpack = True, usecols=1)
decs = np.loadtxt('../Data/1.OSCDecDeg.txt',delimiter='" ',unpack = True, usecols = (1))
types = np.loadtxt('../Data/1.OSCCleaned.csv', dtype=str, unpack=True, usecols=3)

In [3]:
#Checks if Supernovae are in an image and reports back their x & y coords if in map

def check_in_image(SNras, SNdecs, SNnames, SNtypes, image, maptype = "MUSE"):
    
    hdulist  = pyfits.open(image)
    
    if maptype == "MUSE":
        map = hdulist["HA6562_FLUX"].data
        #emap = hdulist["HA6562_FLUX_ERR"].data     
        wcs = WCS(hdulist["HA6562_FLUX"].header, naxis=2)
        naxis = wcs._naxis #size of image
        naxis1 = hdulist["HA6562_FLUX"].header['NAXIS1']
        naxis2 = hdulist["HA6562_FLUX"].header['NAXIS2']
    else:
        map = hdulist[0].data
        #emap = "find a way to use RMS from galaxy for these"
        wcs = WCS(hdulist[0].header, naxis=2)
        #beamsize = hdulist[0].header["BEAM"]
        naxis = wcs._naxis #size of image
        naxis1 = hdulist[0].header['NAXIS1']
        naxis2 = hdulist[0].header['NAXIS2']
        
    coords_arr = np.column_stack((SNras, SNdecs)) # ras and decs now [ra,dec]
    pix_x, pix_y = wcs.wcs_world2pix(SNras,SNdecs,0)

    #use world coordinates of all SNe to see if any fall in image (our version of footprint_contains)

    is_in_x = (pix_x >= 0) & (pix_x <= naxis[0]-1) #because of 0-indexing
    is_in_y = (pix_y >= 0) & (pix_y <= naxis[1]-1)
             
    #get the name, ra, and dec of the SNe that fall in image
    #boolean array indexing (gives back array of Trues and Falses)
    #we are pulling out the SNe that are True and assigning them to own arrays
    name_in_image = np.array(SNnames)[is_in_x & is_in_y]
    type_in_image = np.array(SNtypes)[is_in_x & is_in_y]
    ra_in_image = np.array(SNras)[is_in_x & is_in_y]
    dec_in_image = np.array(SNdecs)[is_in_x & is_in_y]

    x_coord = np.array(pix_x)[is_in_x & is_in_y]
    y_coord = np.array(pix_y)[is_in_x & is_in_y]
    
    return(name_in_image, type_in_image, ra_in_image, dec_in_image, x_coord, y_coord, map, naxis1, naxis2)
    #print(name_in_image, type_in_image, ra_in_image, dec_in_image, x_coord, y_coord)


In [4]:
# Build MUSE Catalogue That has all OSC entries within PHANGS-MUSE footprint

with open('../Data/2.SNeHa_MUSE_Cat.txt', 'w') as text_file:
    print('#Galaxy,Supernova,Ra,Dec,MUSE_xCoord,MUSE_yCoord,Type,MUSE_Intensity', file = text_file)

for i in range(len(galaxies)):
    if (MUSEmaps[i] != "" and os.path.isfile(MUSEmaps[i])):
    
        found_name, found_type, found_ra, found_dec, x_coord, y_coord, map, naxis1, naxis2 = check_in_image(ras, decs, SNe, types, MUSEmaps[i])
        intensity = []
    
        for k in range(len(found_type)):

            xVal = int(round(x_coord[k]))
            yVal = int(round(y_coord[k]))
            if (yVal <= naxis2) and (xVal <= naxis1):
                intensity.append(map[yVal, xVal])

            else: 
                intensity.append(np.nan)


        with open('../Data/2.SNeHa_MUSE_Cat.txt', 'a') as text_file:
            for j in range(len(found_name)):
                print(galaxies[i], ",", found_name[j],",", found_ra[j], ",",found_dec[j],",", x_coord[j],",",y_coord[j],",", found_type[j],",", intensity[j], file = text_file)


In [5]:
# Build ESO-MPG Catalogue That has all OSC entries within PHANGS-ESO-MPG footprint

with open('../Data/2.SNeHa_ESO_Cat.txt', 'w') as text_file:
    print('#Galaxy,Supernova,Ra,Dec,ESO_xCoord,ESO_yCoord,Type,ESO_Intensity', file = text_file)

for i in range(len(galaxies)):
    if (ESOmaps[i] != "" and os.path.isfile(ESOmaps[i])):
    
        found_name, found_type, found_ra, found_dec, x_coord, y_coord, map, naxis1, naxis2 = check_in_image(ras, decs, SNe, types, ESOmaps[i], maptype="ESO") 
        intensity = []
    
        for k in range(len(found_type)):

            xVal = int(round(x_coord[k]))
            yVal = int(round(y_coord[k]))
            if (yVal <= naxis2) and (xVal <= naxis1):
                intensity.append(map[yVal, xVal])

            else: 
                intensity.append(np.nan)


        with open('../Data/2.SNeHa_ESO_Cat.txt', 'a') as text_file:
            for j in range(len(found_name)):
                print(galaxies[i], ",", found_name[j],",", found_ra[j], ",",found_dec[j],",", x_coord[j],",",y_coord[j],",", found_type[j],",", intensity[j], file = text_file)


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


In [14]:
# Turn files into data structure and clean

ESOdataFile = '../Data/2.SNeHa_ESO_Cat.txt'
MUSEdataFile = '../Data/2.SNeHa_MUSE_Cat.txt'
ESOTable = Table.read(ESOdataFile, format='ascii') 
MUSETable = Table.read(MUSEdataFile, format='ascii') 
database = join(ESOTable, MUSETable, keys=('Galaxy', "Supernova", "Ra", "Dec", "Type"), join_type="outer")
database = join(database, Data, keys=('Galaxy'))

#database
ascii.write(database, '../Data/2.SNeHaMasterCat.csv', format='csv', fast_writer=False, overwrite=True)  


In [15]:

SN_name = [x[2:-2] for x in database["Supernova"]]

# Here we are going to filter out our undesired OSC entries:
    # One is a double counted SN named "PTSS-19clju", any LRNs or LBVs or IIns, and any candidates

flags = []
flag1 = database['Supernova'] == "PTSS-19clju"

flag2 = []

for i in range(len(SN_name)):
    if ("LRN" in database['Type'][i] or "LBV" in database['Type'][i] or "IIn" in database['Type'][i] or "Candidate" in database['Type'][i]):
        flag2.append(True)
    else:
        flag2.append(False)

for i in range(len(flag1)):
    if flag1[i] == True or flag2[i] == True:
        flags.append(True)
    else:
        flags.append(False)
        
flagsArr = np.array(flags)

c = Column(data=flags, name='Removed')
database.add_column(c)
ascii.write(database, '../Data/2.SNeHaMasterCat.csv', format='csv', fast_writer=False, overwrite = True) 

cleanData = database[flagsArr==False]

cleanData

ascii.write(cleanData, '../Data/2.SNeHaMasterCatClean.csv', format='csv', fast_writer=False, overwrite = True)  

In [16]:
cleanData

Galaxy,Supernova,Ra,Dec,ESO_xCoord,ESO_yCoord,Type,ESO_Intensity,MUSE_xCoord,MUSE_yCoord,MUSE_Intensity,ESO-MPG_Hasub_wcomb_corr,MUSEResolution,MUSEmap,Removed
str8,str134,float64,float64,float64,float64,str12,float64,float64,float64,float64,str85,float64,str80,bool
NGC0628,PS15blm,24.258708333333335,15.777000000000001,1253.8697326978763,2391.66186884759,unclassified,6.3913784,--,--,--,/home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC0628_Hasub_flux_corrs.fits,0.92,/data/fourier/sun.1608/PHANGS/MUSE/DR2.1/MUSEDAP/copt/NGC0628-0.92asec_MAPS.fits,False
NGC0628,SN2002ap,24.09940416666667,15.753649074074074,3577.0564594599123,2037.1622175556208,Ic,-0.17532486,--,--,--,/home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC0628_Hasub_flux_corrs.fits,0.92,/data/fourier/sun.1608/PHANGS/MUSE/DR2.1/MUSEDAP/copt/NGC0628-0.92asec_MAPS.fits,False
NGC0628,SN2003gd,24.17770555555555,15.738966666666665,2434.990386106029,1814.759548435055,II,0.55219537,--,--,--,/home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC0628_Hasub_flux_corrs.fits,0.92,/data/fourier/sun.1608/PHANGS/MUSE/DR2.1/MUSEDAP/copt/NGC0628-0.92asec_MAPS.fits,False
NGC0628,SN2013ej,24.200666666666667,15.758611111111112,2100.1951571946192,2112.559714338542,II,0.09700711,243.19103784243157,305.29329959411643,nan,/home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC0628_Hasub_flux_corrs.fits,0.92,/data/fourier/sun.1608/PHANGS/MUSE/DR2.1/MUSEDAP/copt/NGC0628-0.92asec_MAPS.fits,False
NGC1087,SN1995V,41.61152777777777,-0.4987861111111111,648.7610232419815,747.9909198104285,II,5.0002465,183.09993631014947,450.72004174402633,2663.371834437333,/home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC1087_Hasub_flux_corrs.fits,0.92,/data/fourier/sun.1608/PHANGS/MUSE/DR2.1/MUSEDAP/copt/NGC1087-0.92asec_MAPS.fits,False
NGC1300,LSQ14dtr,49.96679166666666,-19.478694444444443,841.190251571016,472.6653386793862,Ia,-0.06859025,--,--,--,/home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC1300_Hasub_flux_corrs.fits,0.89,/data/fourier/sun.1608/PHANGS/MUSE/DR2.1/MUSEDAP/copt/NGC1300-0.89asec_MAPS.fits,False
NGC1365,SN2012fr,53.40057916666666,-36.12676944444445,3041.428195030292,3237.1579264657776,Ia,0.36029622,756.0731074267953,702.5736600562143,69.29969489043229,/home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC1365_Hasub_flux_corrs.fits,1.15,/data/fourier/sun.1608/PHANGS/MUSE/DR2.1/MUSEDAP/copt/NGC1365-1.15asec_MAPS.fits,False
NGC1365,SN1957C,53.38349999999999,-36.11770833333333,3250.4029666345923,3374.662468105187,unclassified,2.834359,1004.4155369313344,865.6486483514018,1697.5272118916866,/home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC1365_Hasub_flux_corrs.fits,1.15,/data/fourier/sun.1608/PHANGS/MUSE/DR2.1/MUSEDAP/copt/NGC1365-1.15asec_MAPS.fits,False
NGC1365,SN1983V,53.381876388888884,-36.14859166666667,3270.6237389153885,2906.610095778981,Ic,9.090737,1027.9095105186898,309.74396493992725,5743.3576143866185,/home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC1365_Hasub_flux_corrs.fits,1.15,/data/fourier/sun.1608/PHANGS/MUSE/DR2.1/MUSEDAP/copt/NGC1365-1.15asec_MAPS.fits,False


In [21]:
for i in range(len(galaxies)):
    print(data["Galaxy"][i], data["ESO-MPG_Hasub_wcomb_corr"][i])

NGC0628 /home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC0628_Hasub_flux_corrs.fits
NGC0628 /home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC0628_Hasub_flux_corrs.fits
NGC0628 /home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC0628_Hasub_flux_corrs.fits
NGC0628 /home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC0628_Hasub_flux_corrs.fits
NGC1087 /home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC1087_Hasub_flux_corrs.fits
NGC1300 /home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC1300_Hasub_flux_corrs.fits
NGC1365 /home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC1365_Hasub_flux_corrs.fits
NGC1365 /home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC1365_Hasub_flux_corrs.fits
NGC1365 /home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC1365_Hasub_flux_corrs.fits
NGC1365 /home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC1365_Hasub_flux_corrs.fits
NGC1433 /home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC1433

In [23]:
# Plot ESO Galaxy Files with SNe

data = ascii.read('../Data/2.SNeHaMasterCatClean.csv')

for i in range(len(data["Galaxy"])):
    
    if (data["ESO-MPG_Hasub_wcomb_corr"][i] != "" and os.path.isfile(data["ESO-MPG_Hasub_wcomb_corr"][i])):     
    
        found_name, found_type, found_ra, found_dec, x_coord, y_coord, map, naxis1, naxis2 = check_in_image(data['Ra'], data['Dec'], data['Supernova'], data['Type'], data["ESO-MPG_Hasub_wcomb_corr"][i], maptype = "ESO")
    
        plt.figure(figsize=(7, 7))
        logmap = np.log10(map)
        plt.imshow(logmap, origin='lower', interpolation='nearest', zorder=1)

        for k in range(len(found_name)):

            string = found_name[k]

            if (len(found_type[k])>1):
                if found_type[k][1] == "I":
                    #colorCode = 'chartreuse'
                    colorCode = 'black'
                    mkr = '*'
                elif found_type[k][1] == "b" or found_type[k][1] == "c":
                    #colorCode = 'salmon'
                    colorCode = 'darkblue'
                    mkr = '^'
                elif found_type[k][1] == "a":
                    #colorCode = 'deeppink'
                    colorCode = 'indigo'
                    mkr = 'o'
                else:
                    colorCode = 'slategrey'
                    mkr = ''
            else:
                #colorCode = 'cyan'
                colorCode = 'maroon'
                mkr='s'

            plt.plot(x_coord[k], y_coord[k], marker=mkr, ms=10, color=colorCode, zorder=10)
            plt.text(x_coord[k]-20, y_coord[k]+8, string, fontsize=12, color=colorCode)

        plt.axis('off')
        plt.title(data["Galaxy"][i], fontsize=24, fontweight='bold')
        plt.tight_layout()
        plt.savefig('../Figures/2.ESO_Galaxy_Images/' + data["Galaxy"][i] + '.png')
        if (len(found_name) > 0):
            plt.savefig('../Figures/2.ESO_SNe_Galaxy_Images/' + data["Galaxy"][i] + '.png')
        plt.close()

<ipython-input-23-879722430c28>:12: RuntimeWarning: invalid value encountered in log10
  logmap = np.log10(map)
<ipython-input-23-879722430c28>:12: RuntimeWarning: divide by zero encountered in log10
  logmap = np.log10(map)
<ipython-input-23-879722430c28>:45: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  plt.tight_layout()


In [24]:
# Plot MUSE Galaxy Files with SNe

for i in range(len(data["Galaxy"])):
    
    if (data["MUSEmap"][i] != "" and os.path.isfile(data["MUSEmap"][i])):     
    
        found_name, found_type, found_ra, found_dec, x_coord, y_coord, map, naxis1, naxis2 = check_in_image(data['Ra'], data['Dec'], data['Supernova'], data['Type'], data["MUSEmap"][i])
    
        plt.figure(figsize=(7, 7))
        logmap = np.log10(map)
        plt.imshow(logmap, origin='lower', interpolation='nearest', zorder=1)

        for k in range(len(found_name)):

            string = found_name[k]

            if (len(found_type[k])>1):
                if found_type[k][1] == "I":
                    #colorCode = 'chartreuse'
                    colorCode = 'black'
                    mkr = '*'
                elif found_type[k][1] == "b" or found_type[k][1] == "c":
                    #colorCode = 'salmon'
                    colorCode = 'darkblue'
                    mkr = '^'
                elif found_type[k][1] == "a":
                    #colorCode = 'deeppink'
                    colorCode = 'indigo'
                    mkr = 'o'
                else:
                    colorCode = 'slategrey'
                    mkr = ''
            else:
                #colorCode = 'cyan'
                colorCode = 'maroon'
                mkr='s'

            plt.plot(x_coord[k], y_coord[k], marker=mkr, ms=10, color=colorCode, zorder=10)
            plt.text(x_coord[k]-20, y_coord[k]+8, string, fontsize=12, color=colorCode)

        plt.axis('off')
        plt.title(data["Galaxy"][i], fontsize=24, fontweight='bold')
        plt.tight_layout()
        plt.savefig('../Figures/2.MUSE_Galaxy_Images/' + data["Galaxy"][i] + '.png')
        if (len(found_name) > 0):
            plt.savefig('../Figures/2.MUSE_SNe_Galaxy_Images/' + data["Galaxy"][i] + '.png')
        plt.close()

<ipython-input-24-279dfdabffea>:10: RuntimeWarning: divide by zero encountered in log10
  logmap = np.log10(map)
